# 1.- Intervenciones partidos políticos - Descarga del corpus

<a target="_blank" href="https://colab.research.google.com/github/Chiriviki/congreso/blob/8e68b19b0889865baa767c4628d2990b06993450/1.-%20Descarga_corpus.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

El dataset se se compone de la unión de varias conjuntos. Por un lado tenemos las intervenciones de cada parlamentario en los diarios de sesiones y por otro tenemos las tabla de pertenencia a los partidos políticos. Es la primera parte la que conlleva más trabajo. Para la segunda simplemente se han extraído de la [página de wikipedia](https://es.wikipedia.org/wiki/Anexo:Diputados_de_la_XIV_legislatura_de_Espa%C3%B1a) el listado de parlamentarios de la legislatura. [Esta herramienta](https://wikitable2csv.ggor.de/) permite desargarlas en formato csv.

## Diseño - Tareas

Para extraer las intervenciones se ha dividio el trabajo en las siguientes fases:

1. Descarga de las intervenciones: método para solicitar de forma automatizada al servidor el texto de un diario de sesiones.
2. Limpieza de intervenciones: dada la respuesta del servidor, en esta fase se limpian las etiquetas que no se consideran útiles. El resultado es el texto junto con algunos metadatos.
3. Extracción de intervenciones: en esta fase se separan las intervenciones de cada parlamentario, y se obtiene el nombre del parlamentario. 

Una vez definidas se unen todas para conseguir todas las intervenciones de una determinada legislatura.


## Descarga de intervenciones

La solicitud se realizan a la URL 'https://www.congreso.es/publicaciones-organo'. El método POST permite los siguientes parámetros:

- 'p_p_id': 'publicaciones',
- 'p_p_lifecycle': '0',
- 'p_p_state': 'normal',
- 'p_p_mode': 'view',
- '_publicaciones_mode': 'mostrarTextoIntegro',
- '_publicaciones_legislatura': '',
- '_publicaciones_texto': '',
- '_publicaciones_id_texto': documento,



A nosotros solo nos interesa:
- _publicaciones_legislatura: Legislatura en números romanos.
- _publicaciones_id_texto: Identificador cve del documento.

Todos los documentos del congreso incluyen un identificador (cve) que siguen siempre una misma estructura:
`{DOCUMENTO_ORGANO}-{LEGISLATURA}-{TIPO_SESION}-{ID}.CODI`

Donde:
- DOCUMENTO_ORGANO: Los diarios de sesiones del congreso de los diputados tiene en esta sección DSCD.
- LEGISLATURA: Legislatura
- TIPO_SESION: Para los plenos utiliza "PL", para las comisiones "CO" y para las comisiones de investigación "CI".
- ID es un número entero por orden cronológico en la legislatura.

Para que el servidor permita descargar múltiples archivos es necesario establecer algunos valores de la solicitud. La siguiente función descarga un documento dada la legislatura y el identificador del documento.


In [ ]:
import requests

def request_document(legislatura, documento):
    
    cookies = {
        'COOKIE_SUPPORT': 'true',
        'GUEST_LANGUAGE_ID': 'es_ES',
        'mycookieaceptthx': 'eyJtYW5kYXRvcnkiOnRydWUsInN0YWRpc3RpY3MiOmZhbHNlfQ==',
        'JSESSIONID': 'WrhE0i3bOfbukUPJ_FiUJmU2rjB7EsjPjSt9oci8.cgdpjbnode1pro',
        'LFR_SESSION_STATE_20104': '1673524465260',
    }

    headers = {
        'authority': 'www.congreso.es',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-language': 'es-ES,es;q=0.9,en;q=0.8',
        'cache-control': 'max-age=0',
        # 'cookie': 'COOKIE_SUPPORT=true; GUEST_LANGUAGE_ID=es_ES; mycookieaceptthx=eyJtYW5kYXRvcnkiOnRydWUsInN0YWRpc3RpY3MiOmZhbHNlfQ==; JSESSIONID=WrhE0i3bOfbukUPJ_FiUJmU2rjB7EsjPjSt9oci8.cgdpjbnode1pro; LFR_SESSION_STATE_20104=1673524465260',
        'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    params = {
        'p_p_id': 'publicaciones',
        'p_p_lifecycle': '0',
        'p_p_state': 'normal',
        'p_p_mode': 'view',
        '_publicaciones_mode': 'mostrarTextoIntegro',
        '_publicaciones_legislatura': legislatura,
        '_publicaciones_texto': '',
        '_publicaciones_id_texto': documento,
    }

    response = requests.get('https://www.congreso.es/publicaciones-organo', params=params, cookies=cookies, headers=headers)
    
    return response

Si probamos con el pleno del congreso 22, obtenemos el siguiente resultado.

In [ ]:
rpnse = request_document("XIV", "DSCD-14-PL-1.CODI.")
rpnse

<Response [200]>

In [ ]:
rpnse.text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t<!DOCTYPE html>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html class="ltr" dir="ltr" lang="es-ES">\n\n<head>\n\t<title>DS. Congreso de los Diputados, Pleno y Dip. Perm., núm. 1, de 03/12/2019 - Congreso de los Diputados</title>\n\n\t<meta content="initial-scale=1.0, width=device-width" name="viewport" />\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n\n\t<script type="text/javascript">\n\t\t\n\t\tvar checkSavedCookies = document.cookie.split(\';\');\n\t\tvar acceptedStadistics = false;\n\t\tfor(var i = 0; i < checkSavedCookies.length; i++) {\n\t\t  \tvar par = checkSavedCookies[i].trim().split(\'=\');\n\t\t  \tif(par[0] == \'mycookieaceptthx\' ){\n\t\t  \t\tvar cookieVal = par[1];\n\t\t  \t\tvar objAcceptedCookies = JSON.parse(atob(cookieVal));\n\t\t  \t\tif( objAcceptedCookies[\'stadistics\'] ){\n\t\t  \t\t\taccepted

### Limpieza

Hemos podido ver a maraña de etiquetas html que devuelve el servidor. Las etiquetas que nos iteresan son: 

- `<dov class:"datos1">`: Incluye el organismo, sesión y fecha (metadatos).
- `<p class:"textoCompleto">`: Incluye el texto transcrito del diario de sesiones. Incluye algunas etiquetas de salto de línea y enlaces que se pueden eliminar.



A continuación se define un método para extraer las dos etiquetas y eliminar las etiuetas inútiles. El método mantiene saltos de línea que se puede considerar eliminar posteriormente.

Si el documento no existe, el servidor si devuelve una respuesta pero estas dos etiquetas principales no se encuentran. El método devuelve None en tal caso.

In [ ]:
from bs4 import BeautifulSoup

def extract_text(texto):
    
    # Extrae textoCompleto
    soup = BeautifulSoup(texto, 'html.parser')    
    datos1 = soup.find("div", {"class":"datos1"})
    
    if datos1:        
        textoCompleto = soup.find("p", {"class":"textoCompleto"})

        # ELimina enlaces
        for tag in textoCompleto.findAll("a"):
            tag.decompose()

        # Concatena el resto del texto mediante espacios
        texto_limpio = "\n".join([t for t in textoCompleto.stripped_strings])
        return {"datos":datos1.text, "texto":texto_limpio}
    
    else:
        return None

Si lo probamos con el documento anterior obtenemos el siguiente resultado:

In [ ]:
limpio = extract_text(rpnse.text)
limpio

{'datos': 'DS. Congreso de los Diputados, Pleno y Dip. Perm., núm. 1, de 03/12/2019',
 'texto': "ORDEN DEL DÍA:\nConstitución de la Mesa de Edad ...\nReal decreto de convocatoria de elecciones ...\nRelación alfabética de señoras y señores diputados electos ...\nRecursos contencioso-electorales interpuestos ...\nElección de la Mesa del Congreso de los Diputados ...\nJuramento o promesa de acatamiento de la Constitución ...\nDiscurso de la señora presidenta del Congreso de los Diputados ...\nSUMARIO\nSe abre la sesión a las diez de la mañana.\nConstitución de la Mesa de Edad ...\nSe constituye la Mesa de Edad, formada por los diputados electos don Agustín Zamarrón Moreno, como presidente, y doña Marta Rosique i Saltor y doña Lucía Muñoz Dalda, como secretarias.\nReal decreto de convocatoria de elecciones ...\nLa señora secretaria de la Mesa de Edad (Rosique i Saltor) da lectura al artículo 5 del Real Decreto 551/2019, de 24 de septiembre, de disolución del Congreso de los Diputados y del

### Extraer intervenciones

En el diario de sesiones, cada vez que un parlamentario toma la palabra se precede de un texto con formato similar:

*La señora VICEPRESIDENTA (Pastor Julián): (...)*

*El señor BEL ACCENSI: (...)*

*El señor CAPDEVILA I ESTEVE: (...)*


Se puede ver que cumple una serie de reglas.
- Siempre empieza por salto de línea y "El señor"/"La señora".
- Si es un parlamentario sin puesto en el gobierno o congreso, continúa con sus apellidos en mayúsculas.
- Si tiene puesto, este aparece en mayúsculas y a continuación sus apellidos entre parentesis. El presidente/a del congreso no incluye su nombre.
- Siempre acaba con ":".

Todo lo que hay entre estas citas es el discurso del parlamentario. Por lo tanto, extraer la intervención se compone de dos pasos:

1. Primero se identifican las introducciones y se extrae información de ellas (apellidos del parlamentario) y índice en el texto.
2. Extraer el texto que hay entre dos intervenciones y asignarlo al parlamentario correspondiente.


A continuación se define una función que incluye estos dos conceptos.


La expresión regular utilizada es:

```\n((El señor)|(La señora)) ([\s\-,\w\d]*){1}( \(([^\)]*)\))?:```


In [ ]:
import re

def extract_intervenciones(texto):
    
    # Extrae las introducciones de cada intervención    
    regexp = re.compile(r"\n((El señor)|(La señora)) (?P<name1>[\s\w\d\-,]*){1}( \((?P<name2>[^\)]*)\))?:")
    matches = regexp.finditer(texto)    
    intervenciones = []
    for match in matches:
        groups = match.groupdict()
        intervenciones.append({"name1":groups['name1'], "name2":groups['name2'], "span":match.span()})
        

    # Añade el texto tomando los índices del match y el siguiente
    for i in range(0, len(intervenciones)-1):

        leftindex = intervenciones[i]["span"][1]
        rightindex = intervenciones[i+1]["span"][0]

        intervenciones[i]["texto"] = texto[leftindex:rightindex]

    intervenciones[-1]["texto"] = texto[intervenciones[-1]["span"][1]:]
    
    return intervenciones

 El resultado es una lista con un diccionario por intervención. Este incluye los siguientes campos:
- `name1`: Parte de la introducción que aparece en mayúsculas. Apellidos o puesto si lo tiene.
- `name2`: Apellidos en caso que tenga puesto.
- `span`(a eliminar): índices de la introducción utilizados para obtener el texto.
- `texto`: texto de la intervención

Probamos con el texto que estamos utilizando hasta ahora.

In [ ]:
intervenciones = extract_intervenciones(limpio["texto"])
intervenciones[:10]

[{'name1': 'PRESIDENTE DE LA MESA DE EDAD',
  'name2': 'Zamarrón Moreno',
  'span': (5107, 5165),
  'texto': ' Señorías, se abre la sesión.\nEn virtud de lo dispuesto en el artículo 2 del Reglamento del Congreso de los Diputados, la Mesa de Edad ha quedado constituida por el diputado electo de mayor edad de los presentes, don Agustín Zamarrón\nMoreno -es decir, yo mismo-, como presidente, y por las dos más jóvenes como secretarias, a saber, doña Marta Rosique i Saltor y doña Lucía Muñoz Dalda, según los datos que constan en la Cámara.\nSeñorías, el artículo 99 de nuestra Constitución expone el artificioso modo para el nombramiento de presidente de Gobierno, dando inicio a un proceso que culmina en un Gobierno legítimo y pleno en sus atribuciones; al hacerlo determina la\ngrave responsabilidad de los intervinientes en el proceso, en lo que afecta a la responsabilidad de las señoras y señores diputados y de la Cámara en pleno. Cierto es que responsabilidad, mérito y empeño es distinto pa

## Juntarlo todo

Ahora es necesario unir todo para obtener todos las intervenciones de una legislatura. Para ello es necesario que se definan dos funciones.

- Una que dada una legislatura y un documento realice todas las tareas anteriores.
- Otra que defina un método para generar los distintos identificadores de documentos dada una legislatura. Obtenga las intervenciones y guarde el resultado.

A continuación se definen ambas:

In [ ]:
def get_documento(legislatura, id_documento):

    # Solicita al servidor
    rpnse = request_document(legislatura, id_documento)
    
    # Si la respuesta no es OK lanza eceptcion
    if not rpnse.ok:
        raise Exception(f"No se pudo conectar con el servidor Error {rpnse.status_code}")
        
    # Extrae texto y metadatos
    ds = extract_text(rpnse.text)
    
    if ds is None:
        raise Exception(f"No se ha detectado un documento válido. (CVE: {id_documento})")
        
    # Extrae intervenciones

    ds["intervenciones"] = extract_intervenciones(ds["texto"])
    del ds["texto"]
    
    return ds


carpeta = "dataset\corpus_v1"

get_documento(carpeta, "DSCD-14-PL-140.CODI.")

{'datos': 'DS. Congreso de los Diputados, Pleno y Dip. Perm., núm. 140, de 24/11/2021',
 'intervenciones': [{'name1': 'VICEPRESIDENTE',
   'name2': 'Rodríguez Gómez de Celis',
   'span': (11324, 11376),
   'texto': ' Muy buenos días. Se reanuda la sesión.\nSeñorías, habiendo finalizado la votación de las enmiendas hasta la sección 16 incluida, en relación con el dictamen de Comisión relativo al proyecto de ley de Presupuestos Generales del Estado para el año 2022, les anuncio que han resultado\naprobadas las siguientes enmiendas: al título VI, enmienda transaccional número 1 a la enmienda 5209, del Grupo Parlamentario Mixto, señor Quevedo Iturbe; a las disposiciones adicionales transitorias, derogatorias y finales, enmiendas\ntransaccionales de la 2 a la 12; a la sección 28, Ministerio de Ciencia e Innovación, enmiendas transaccionales 13 a 15, y a la sección 16,\nMinisterio del Interior, enmiendas transaccionales 16 y 17. Adicionalmente, ha resultado aprobada la enmienda de corrección

In [ ]:
import os
import json
from tqdm import tqdm

def download_corpus(legislatura_num, legislatura_rom, id_ultimo_doc, folder="."):
    
    
    # Si no existe el directorio lo crea
    folder_name = os.path.join(folder, legislatura_rom)
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    for id_doc in tqdm(range(1, id_ultimo_doc+1)):    
        doc_name = f"DSCD-{legislatura_num}-PL-{id_doc}.CODI."
        doc = get_documento(legislatura_rom, doc_name)
        
        filename = os.path.join(folder_name, f"{id_doc}.json")
        
        with open(filename, "w", encoding="latin1") as f:
            json.dump(doc, f, ensure_ascii=False)
    
# Prueba con 3 documentos
download_corpus(14, carpeta, 3)

100%|██████████| 3/3 [00:07<00:00,  2.38s/it]


In [ ]:
with open(os.path.join(carpeta, "3.json"), "r", encoding="latin1") as f:
    data= json.load(f)

data

{'datos': 'DS. Congreso de los Diputados, Pleno y Dip. Perm., núm. 3, de 05/01/2020',
 'intervenciones': [{'name1': 'PRESIDENTA',
   'name2': None,
   'span': [2295, 2317],
   'texto': ' Buenos días, señorías. Reanudamos la sesión.\nContinuamos con los turnos de intervención de los grupos parlamentarios y finalizadas las intervenciones se procederá a la votación, que, como saben, será pública por llamamiento, de acuerdo con los artículos 85 y 86 del Reglamento de la\nCámara.\nPor el Grupo Parlamentario Euskal Herria Bildu, tiene la palabra la señora Aizpurua Arzallus.'},
  {'name1': 'AIZPURUA ARZALLUS',
   'name2': None,
   'span': [2702, 2731],
   'texto': ' Gracias, señora presidenta. Egun on. Buenos días.\nSeñor Sánchez, permítame en primer lugar que le recuerde algunas de las ideas que expuse en la anterior sesión de investidura y que apunte también algunas cosas que han sucedido desde entonces hasta hoy. Dijimos que en el momento histórico\nde crisis generalizada que vivimos estas

Ahora comprobamos cual es el identificador del último pleno y ejecutamos para todos los de la legislatura.

A fecha 06/03/2023 es el 248. Si se ejecuta correctamente se da por concluida la creación del corpus

In [ ]:
download_corpus(14, carpeta, 248)

100%|██████████| 248/248 [12:34<00:00,  3.04s/it]


# Conclusiones

En este cuaderno se han descargado todos los diarios de sesiones de plenos del congreso de los diputados de la XIV legislatura. Cada DS se ha dividido en intervenciones y extraído metadatos del parlamentario. Cada diario de sesiones se encuentra disponible en formato JSON. Sigue la siguiente estructura:
- datos: metadatos del pleno.
- intervenciones: lista de intervenciones:
    - name1: Nombre del parlamentario o cargo dentro del congreso o gobierno
    - name2: Nombre del parlamentario si tiene cargo.
    - span: indice de los caracteres de inicio y fin que del texto de la intervención.
    - texto: texto de la intervención